# Limpieza del dataset de bienes raíces

Este es un conjunto de datos (dataset) reales que fue descargado usando técnicas de web scraping. El archivo contiene registros de **Fotocasa**, el cual es uno de los sitios más populares de bienes raíces en España. Contiene miles de datos de casas reales publicadas en la web www.fotocasa.com.

El dataset fue descargado hace algunos años y en ningún caso se obtuvo beneficio económico de ello.

Tu objetivo es extraer tanta información como sea posible con el conocimiento que tienes hasta ahora de ciencia de datos.

In [1]:
import pandas as pd

#### Ejercicio 00

Lee el dataset data/real_estate.csv e intenta visualizar la tabla (★☆☆)

In [ ]:
# Este archivo CSV contiene puntos y comas en lugar de comas como separadores
df = pd.read_csv('../data/real_estate.csv', sep=';')
df

## Trabajando con un DataFrame

#### Ejercicio 01

¿Cuál es la casa más cara del dataset? (★☆☆)

Imprime la dirección y el precio de la casa seleccionada. Para visualizar el resultado utiliza un f-string. Por ejemplo:

```py
f'La casa más cara se encuentra en la dirección: {address} y su precio es {price} €'
```

In [ ]:
import re
import numpy as np
import pandas as pd

def _norm(s: str) -> str:
    s = str(s).strip().lower()
    s = re.sub(r"[áàä]", "a", s)
    s = re.sub(r"[éèë]", "e", s)
    s = re.sub(r"[íìï]", "i", s)
    s = re.sub(r"[óòö]", "o", s)
    s = re.sub(r"[úùü]", "u", s)
    s = s.replace("ñ", "n")
    s = re.sub(r"[^a-z0-9]+", "", s)
    return s

def find_col(df: pd.DataFrame, aliases) -> str:
    """Devuelve el nombre real de la primera columna que coincida con los alias dados."""
    m = {_norm(c): c for c in df.columns}
    for a in aliases:
        if _norm(a) in m:
            return m[_norm(a)]
    raise KeyError(f"No encontré ninguna de estas columnas: {aliases}")

# Detectamos columnas típicas del dataset
COL_PRICE = find_col(df, ["price", "precio", "price_eur", "precio_eur", "precio€", "price€"])
COL_ADDR  = find_col(df, ["address", "direccion", "location"])

def to_float_eur(x):
    """Convierte valores tipo '350.000,50 €' o '350,000.50' a float robustamente."""
    if pd.isna(x):
        return np.nan
    s = str(x).strip()
    s = s.replace("€", "").replace("EUR", "").strip()
    # Si tiene punto y coma, asumimos formato europeo: '.' miles y ',' decimales
    if "." in s and "," in s:
        s = s.replace(".", "").replace(",", ".")
    # Si solo tiene comas, las consideramos decimales
    elif "," in s and "." not in s:
        s = s.replace(",", ".")
    # Si solo tiene puntos: ya sería decimal estándar
    # Quitamos cualquier resto no numérico (espacios, etc.)
    s = re.sub(r"[^\d.\-+eE]", "", s)
    try:
        return float(s)
    except:
        return np.nan

# Aseguramos que la columna de precio sea numérica
price_num = df[COL_PRICE].apply(to_float_eur)

# Índice de la fila con precio máximo
idx_max = price_num.idxmax()

address = df.loc[idx_max, COL_ADDR]
price   = price_num.loc[idx_max]

print(f"La casa más cara se encuentra en la dirección: {address} y su precio es {price:,.2f} €".replace(",", "."))

#### Ejercicio 02

¿Cuál es la casa más barata del dataset? (★☆☆)

Imprime la dirección y el precio de la casa seleccionada utilizando f-string

In [ ]:
import re, numpy as np, pandas as pd

# Helpers mínimos (por si no corriste el Ej. 01)
def _norm(s: str) -> str:
    s = str(s).strip().lower()
    s = re.sub(r"[áàä]", "a", s); s = re.sub(r"[éèë]", "e", s)
    s = re.sub(r"[íìï]", "i", s); s = re.sub(r"[óòö]", "o", s)
    s = re.sub(r"[úùü]", "u", s); s = s.replace("ñ", "n")
    return re.sub(r"[^a-z0-9]+", "", s)

def find_col(df: pd.DataFrame, aliases) -> str:
    cols = {_norm(c): c for c in df.columns}
    for a in aliases:
        if _norm(a) in cols:
            return cols[_norm(a)]
    raise KeyError(f"No encontré ninguna de estas columnas: {aliases}")

def to_float_eur(x):
    if pd.isna(x): return np.nan
    s = str(x).strip().replace("€", "").replace("EUR", "").strip()
    if "." in s and "," in s:  # formato EU
        s = s.replace(".", "").replace(",", ".")
    elif "," in s and "." not in s:
        s = s.replace(",", ".")
    s = re.sub(r"[^\d.\-+eE]", "", s)
    try: return float(s)
    except: return np.nan

# Detecto columnas clave
COL_PRICE = find_col(df, ["price", "precio", "price_eur", "precio_eur", "precio€", "price€"])
COL_ADDR  = find_col(df, ["address", "direccion", "location"])

# A número robusto y búsqueda de mínimo
price_num = df[COL_PRICE].apply(to_float_eur)
idx_min   = price_num.idxmin()

address = df.loc[idx_min, COL_ADDR]
price   = price_num.loc[idx_min]

print(f"La casa más barata se encuentra en la dirección: {address} y su precio es {price:,.2f} €".replace(",", "."))

#### Ejercicio 03

¿Cuál es la casa más grande del dataset? (★☆☆)

Imprime la dirección y el área de las casas seleccionadas utilizando f-string

In [ ]:
import re, numpy as np, pandas as pd

# Helpers mínimos (por si no corriste E01/E02)
def _norm(s: str) -> str:
    s = str(s).strip().lower()
    s = re.sub(r"[áàä]", "a", s); s = re.sub(r"[éèë]", "e", s)
    s = re.sub(r"[íìï]", "i", s); s = re.sub(r"[óòö]", "o", s)
    s = re.sub(r"[úùü]", "u", s); s = s.replace("ñ", "n")
    return re.sub(r"[^a-z0-9]+", "", s)

def find_col(df: pd.DataFrame, aliases) -> str:
    cols = {_norm(c): c for c in df.columns}
    for a in aliases:
        if _norm(a) in cols:
            return cols[_norm(a)]
    raise KeyError(f"No encontré ninguna de estas columnas: {aliases}")

def to_float_area(x):
    """Convierte valores tipo '123 m²', '123,5 m2', '123.5' a float."""
    if pd.isna(x): return np.nan
    s = str(x).strip()
    s = s.replace("m²", "").replace("m2", "").replace("metros", "").replace("metros2", "").strip()
    if "." in s and "," in s:          # formato EU: '.' miles, ',' decimales
        s = s.replace(".", "").replace(",", ".")
    elif "," in s and "." not in s:    # solo coma = decimal
        s = s.replace(",", ".")
    s = re.sub(r"[^\d.\-+eE]", "", s)  # limpio cualquier otro símbolo
    try: return float(s)
    except: return np.nan

# Detecto columnas
COL_ADDR = find_col(df, ["address", "direccion", "location"])
COL_SIZE = find_col(df, ["size", "area", "surface", "superficie", "m2", "metroscuadrados", "surface_m2"])

# A número y búsqueda de máximo
size_num = df[COL_SIZE].apply(to_float_area)
idx_max  = size_num.idxmax()

address = df.loc[idx_max, COL_ADDR]
area    = size_num.loc[idx_max]

print(f"La casa más grande está en: {address} y su área es {area:,.2f} m²".replace(",", "."))

#### Ejercicio 04

¿Cuál es la casa más pequeña del dataset? (★☆☆)

In [ ]:
import re, numpy as np, pandas as pd

# Helpers mínimos (compatibles con E01–E03)
def _norm(s: str) -> str:
    s = str(s).strip().lower()
    s = re.sub(r"[áàä]", "a", s); s = re.sub(r"[éèë]", "e", s)
    s = re.sub(r"[íìï]", "i", s); s = re.sub(r"[óòö]", "o", s)
    s = re.sub(r"[úùü]", "u", s); s = s.replace("ñ", "n")
    return re.sub(r"[^a-z0-9]+", "", s)

def find_col(df: pd.DataFrame, aliases) -> str:
    cols = {_norm(c): c for c in df.columns}
    for a in aliases:
        if _norm(a) in cols:
            return cols[_norm(a)]
    raise KeyError(f"No encontré ninguna de estas columnas: {aliases}")

def to_float_area(x):
    """Convierte '123 m²', '123,5 m2', '123.5' → float (m²)."""
    if pd.isna(x): return np.nan
    s = str(x).strip()
    s = (s.replace("m²", "").replace("m2", "").replace("metros", "")
           .replace("metros2", "").replace("㎡", "").strip())
    if "." in s and "," in s:          # Formato europeo
        s = s.replace(".", "").replace(",", ".")
    elif "," in s and "." not in s:    # Solo coma → decimal
        s = s.replace(",", ".")
    s = re.sub(r"[^\d.\-+eE]", "", s)
    try: return float(s)
    except: return np.nan

# Detecto columnas relevantes
COL_ADDR = find_col(df, ["address", "direccion", "location"])
COL_SIZE = find_col(df, ["size", "area", "surface", "superficie", "m2", "metroscuadrados", "surface_m2"])

# A número y búsqueda de mínimo (ignorando 0 y NaN para evitar falsos mínimos)
size_num = df[COL_SIZE].apply(to_float_area).replace(0, np.nan)

idx_min = size_num.idxmin()
address = df.loc[idx_min, COL_ADDR]
area    = size_num.loc[idx_min]

print(f"La casa más pequeña está en: {address} y su área es {area:,.2f} m²".replace(",", "."))

#### Ejercicio 05.

¿El dataset contiene valores no admitidos (NAs)? (★☆☆)

- Muestra el nombre de las filas seguidas por un booleano (`True` o `False`) según contengan o no contengan NAs.
- También muestra el nombre de las columnas seguidas por un booleano (`True` o `False`) según contengan o no contengan NAs.

In [ ]:
import pandas as pd

# Fila a fila → True si la fila tiene al menos un NaN
print("¿Cada fila tiene valores faltantes?\n")
print(df.isna().any(axis=1))

# Columna a columna → True si la columna tiene al menos un NaN
print("\n¿Cada columna tiene valores faltantes?\n")
print(df.isna().any())

#### Ejercicio 06.

Elimina los NAs del dataset, si aplica (★★☆)

Muestra las dimensiones del DataFrame original y del DataFrame después de las eliminaciones.

In [ ]:
import pandas as pd

# Dimensiones antes de eliminar los NaN
shape_before = df.shape

# Eliminamos las filas que contienen al menos un valor NaN
df_clean = df.dropna()

# Dimensiones después de eliminar los NaN
shape_after = df_clean.shape

# Mostramos resultados
print(f"Dimensiones originales: {shape_before}")
print(f"Dimensiones después de eliminar filas con NaN: {shape_after}")

#### Ejercicio 07

¿Cuantas poblaciones (columna level5) contiene el dataset? (★☆☆)

- Muestra una lista con los nombres de las poblaciones
- Muestra el total de las mismas

In [ ]:
import re, pandas as pd

# Función auxiliar para encontrar la columna correcta aunque tenga otro nombre
def _norm(s: str) -> str:
    s = str(s).lower().strip()
    s = re.sub(r"[áàä]", "a", s)
    s = re.sub(r"[éèë]", "e", s)
    s = re.sub(r"[íìï]", "i", s)
    s = re.sub(r"[óòö]", "o", s)
    s = re.sub(r"[úùü]", "u", s)
    s = s.replace("ñ", "n")
    s = re.sub(r"[^a-z0-9]+", "", s)
    return s

def find_col(df: pd.DataFrame, aliases):
    """Devuelve el nombre real de la columna según lista de posibles nombres."""
    cols_norm = {_norm(c): c for c in df.columns}
    for a in aliases:
        if _norm(a) in cols_norm:
            return cols_norm[_norm(a)]
    raise KeyError(f"No se encontró ninguna de las columnas: {aliases}")

# Detectamos la columna de poblaciones (level5 o equivalente)
COL_LEVEL5 = find_col(df, ["level5", "municipio", "poblacion", "localidad", "ciudad", "city", "municipality"])

# Obtenemos las poblaciones únicas, ordenadas alfabéticamente
poblaciones = sorted(df[COL_LEVEL5].dropna().unique().tolist())

# Mostramos resultados
print("Poblaciones registradas en el dataset:\n")
for p in poblaciones:
    print("-", p)

print(f"\nTotal de poblaciones: {len(poblaciones)}")

#### Ejercicio 08

¿Cuál es la media de precios en la población (columna level5) de "Arroyomolinos (Madrid)"? (★★☆)

In [ ]:
import re, numpy as np, pandas as pd

# Funciones auxiliares (por si no corriste los ejercicios anteriores)
def _norm(s: str) -> str:
    s = str(s).lower().strip()
    s = re.sub(r"[áàä]", "a", s)
    s = re.sub(r"[éèë]", "e", s)
    s = re.sub(r"[íìï]", "i", s)
    s = re.sub(r"[óòö]", "o", s)
    s = re.sub(r"[úùü]", "u", s)
    s = s.replace("ñ", "n")
    return re.sub(r"[^a-z0-9]+", "", s)

def find_col(df: pd.DataFrame, aliases):
    cols_norm = {_norm(c): c for c in df.columns}
    for a in aliases:
        if _norm(a) in cols_norm:
            return cols_norm[_norm(a)]
    raise KeyError(f"No se encontró ninguna de las columnas: {aliases}")

def to_float_eur(x):
    """Convierte texto como '350.000,50 €' o '350000' en float."""
    if pd.isna(x):
        return np.nan
    s = str(x).strip().replace("€", "").replace("EUR", "").strip()
    if "." in s and "," in s:
        s = s.replace(".", "").replace(",", ".")
    elif "," in s and "." not in s:
        s = s.replace(",", ".")
    s = re.sub(r"[^\d.\-+eE]", "", s)
    try:
        return float(s)
    except:
        return np.nan

# Detectamos columnas relevantes
COL_PRICE  = find_col(df, ["price", "precio", "price_eur", "precio_eur", "precio€"])
COL_LEVEL5 = find_col(df, ["level5", "municipio", "poblacion", "localidad", "ciudad", "city", "municipality"])

# Convertimos precios a número
df["_price_num"] = df[COL_PRICE].apply(to_float_eur)

# Filtramos por la población objetivo
poblacion = "Arroyomolinos (Madrid)"
media = df.loc[df[COL_LEVEL5] == poblacion, "_price_num"].mean()

# Mostramos resultado
print(f"La media de precios en {poblacion} es de {media:,.2f} €".replace(",", "."))

#### Ejercicio 09.

¿Los precios promedios de "Valdemorillo" y "Galapagar" son iguales? (★★☆)

- Muestra ambos promedios
- Escribe en una celda markdown una conclusión sobre ellos

In [ ]:
import re, numpy as np, pandas as pd

# Helpers por si no corriste celdas previas
def _norm(s: str) -> str:
    s = str(s).lower().strip()
    s = re.sub(r"[áàä]", "a", s); s = re.sub(r"[éèë]", "e", s)
    s = re.sub(r"[íìï]", "i", s); s = re.sub(r"[óòö]", "o", s)
    s = re.sub(r"[úùü]", "u", s); s = s.replace("ñ", "n")
    return re.sub(r"[^a-z0-9]+", "", s)

def find_col(df: pd.DataFrame, aliases):
    cols_norm = {_norm(c): c for c in df.columns}
    for a in aliases:
        if _norm(a) in cols_norm:
            return cols_norm[_norm(a)]
    raise KeyError(f"No se encontró ninguna de las columnas: {aliases}")

def to_float_eur(x):
    if pd.isna(x): return np.nan
    s = str(x).strip().replace("€", "").replace("EUR", "").strip()
    if "." in s and "," in s:  # EU: '.' miles, ',' decimales
        s = s.replace(".", "").replace(",", ".")
    elif "," in s and "." not in s:
        s = s.replace(",", ".")
    s = re.sub(r"[^\d.\-+eE]", "", s)
    try: return float(s)
    except: return np.nan

# Columnas relevantes
COL_PRICE  = find_col(df, ["price", "precio", "price_eur", "precio_eur", "precio€"])
COL_LEVEL5 = find_col(df, ["level5", "municipio", "poblacion", "localidad", "ciudad", "city", "municipality"])

# Precios numéricos
price_num = df[COL_PRICE].apply(to_float_eur)

# Medias por población
m_valdemorillo = price_num[df[COL_LEVEL5] == "Valdemorillo"].mean()
m_galapagar    = price_num[df[COL_LEVEL5] == "Galapagar"].mean()

print(f"Promedio Valdemorillo: {m_valdemorillo:,.2f} €".replace(",", "."))
print(f"Promedio Galapagar  : {m_galapagar:,.2f} €".replace(",", "."))

# Comparación con tolerancia numérica (maneja floats)
iguales = np.isclose(m_valdemorillo, m_galapagar, equal_nan=True)
print("\n¿Son iguales (≈)?", bool(iguales))

#### Ejercicio 10

¿Los promedios de precio por metro cuadrado (precio/m2) de "Valdemorillo" y "Galapagar" son iguales? (★★☆)

> Pista: Crea una nueva columna llamada `pps` (*price per square* o precio por metro cuadrado) y luego analiza los valores.

- Muestra ambos promedios de precio por metro cuadrado
- Escribe en una celda markdown una conclusión sobre ellos

In [ ]:
import re, numpy as np, pandas as pd

# Helpers (por si no corriste las celdas anteriores)
def _norm(s: str) -> str:
    s = str(s).lower().strip()
    s = re.sub(r"[áàä]", "a", s)
    s = re.sub(r"[éèë]", "e", s)
    s = re.sub(r"[íìï]", "i", s)
    s = re.sub(r"[óòö]", "o", s)
    s = re.sub(r"[úùü]", "u", s)
    s = s.replace("ñ", "n")
    return re.sub(r"[^a-z0-9]+", "", s)

def find_col(df: pd.DataFrame, aliases):
    cols_norm = {_norm(c): c for c in df.columns}
    for a in aliases:
        if _norm(a) in cols_norm:
            return cols_norm[_norm(a)]
    raise KeyError(f"No se encontró ninguna de las columnas: {aliases}")

def to_float_eur(x):
    if pd.isna(x): return np.nan
    s = str(x).strip().replace("€", "").replace("EUR", "").strip()
    if "." in s and "," in s:
        s = s.replace(".", "").replace(",", ".")
    elif "," in s and "." not in s:
        s = s.replace(",", ".")
    s = re.sub(r"[^\d.\-+eE]", "", s)
    try: return float(s)
    except: return np.nan

def to_float_area(x):
    if pd.isna(x): return np.nan
    s = str(x).strip().replace("m²", "").replace("m2", "").replace("metros", "").replace("metros2", "").replace("㎡", "").strip()
    if "." in s and "," in s:
        s = s.replace(".", "").replace(",", ".")
    elif "," in s and "." not in s:
        s = s.replace(",", ".")
    s = re.sub(r"[^\d.\-+eE]", "", s)
    try: return float(s)
    except: return np.nan

# Detectamos las columnas relevantes
COL_PRICE  = find_col(df, ["price", "precio", "price_eur", "precio_eur", "precio€"])
COL_LEVEL5 = find_col(df, ["level5", "municipio", "poblacion", "localidad", "ciudad", "city", "municipality"])
COL_SIZE   = find_col(df, ["size", "area", "surface", "superficie", "m2", "metroscuadrados", "surface_m2"])

# Convertimos precios y superficies a valores numéricos
price_num = df[COL_PRICE].apply(to_float_eur)
size_num  = df[COL_SIZE].apply(to_float_area)

# Calculamos precio por m² (evitando divisiones por cero)
df["_price_m2"] = np.where(size_num > 0, price_num / size_num, np.nan)

# Promedios por población
mean_valdemorillo = df.loc[df[COL_LEVEL5] == "Valdemorillo", "_price_m2"].mean()
mean_galapagar    = df.loc[df[COL_LEVEL5] == "Galapagar", "_price_m2"].mean()

# Mostramos resultados
print(f"Precio medio por m² en Valdemorillo: {mean_valdemorillo:,.2f} €".replace(",", "."))
print(f"Precio medio por m² en Galapagar  : {mean_galapagar:,.2f} €".replace(",", "."))

# Comparación con tolerancia numérica
iguales = np.isclose(mean_valdemorillo, mean_galapagar, equal_nan=True)
print("\n¿El precio por m² es igual (≈)?", bool(iguales))

#### Ejercicio 11

¿Cuántas agencia de bienes raíces contiene el dataset? (★★☆)

- Muestra el valor obtenido.

In [ ]:
import re, pandas as pd

# Helper rápido para encontrar la columna correspondiente a la agencia
def _norm(s: str) -> str:
    s = str(s).lower().strip()
    s = re.sub(r"[áàä]", "a", s)
    s = re.sub(r"[éèë]", "e", s)
    s = re.sub(r"[íìï]", "i", s)
    s = re.sub(r"[óòö]", "o", s)
    s = re.sub(r"[úùü]", "u", s)
    s = s.replace("ñ", "n")
    return re.sub(r"[^a-z0-9]+", "", s)

def find_col(df: pd.DataFrame, aliases):
    cols_norm = {_norm(c): c for c in df.columns}
    for a in aliases:
        if _norm(a) in cols_norm:
            return cols_norm[_norm(a)]
    raise KeyError(f"No se encontró ninguna de las columnas: {aliases}")

# Intentamos detectar la columna de agencia inmobiliaria
try:
    COL_AGENCY = find_col(df, ["agency", "real_estate", "inmobiliaria", "agencia", "empresa", "realestate"])
except KeyError:
    COL_AGENCY = None

if COL_AGENCY is None:
    print("⚠️ No se encontró una columna correspondiente a la agencia inmobiliaria.")
else:
    # Contamos las agencias distintas (ignorando NaN)
    n_agencias = df[COL_AGENCY].dropna().nunique()
    print(f"Cantidad de agencias distintas: {int(n_agencias)}")

#### Ejercicio 12

¿Cuál es la población (columna level5) que contiene la mayor cantidad de casas?(★★☆)

- Muestra la población y el número de casas.

In [ ]:
import re, pandas as pd

# Helpers (por si no corriste ejercicios previos)
def _norm(s: str) -> str:
    s = str(s).lower().strip()
    s = re.sub(r"[áàä]", "a", s); s = re.sub(r"[éèë]", "e", s)
    s = re.sub(r"[íìï]", "i", s); s = re.sub(r"[óòö]", "o", s)
    s = re.sub(r"[úùü]", "u", s); s = s.replace("ñ", "n")
    return re.sub(r"[^a-z0-9]+", "", s)

def find_col(df: pd.DataFrame, aliases):
    cols_norm = {_norm(c): c for c in df.columns}
    for a in aliases:
        if _norm(a) in cols_norm:
            return cols_norm[_norm(a)]
    raise KeyError(f"No se encontró ninguna de las columnas: {aliases}")

# Columnas relevantes
COL_LEVEL5 = find_col(df, ["level5", "municipio", "poblacion", "localidad", "ciudad", "city", "municipality"])
# Usamos cualquier columna no nula para contar (dirección suele estar completa)
COL_ADDR   = find_col(df, ["address", "direccion", "location"])

# Conteo por población (incluyendo categoría NaN si existiera)
counts = (
    df.groupby(COL_LEVEL5, dropna=False)[COL_ADDR]
      .count()
      .sort_values(ascending=False)
)

top_pop = counts.index[0]
top_cnt = int(counts.iloc[0])

print(f"Población con mayor cantidad de casas: {top_pop} ({top_cnt} casas)")

# (Opcional) mostrar el top 5
print("\nTop 5 poblaciones por cantidad de casas:")
print(counts.head(5))

---

## Trabajando con un subconjunto del DataFrame

#### Ejercicio 13

Ahora vamos a trabajar con el "cinturón sur" de Madrid.

Haz un subconjunto del DataFrame original que contenga las siguientes poblaciones (columna level5): "Fuenlabrada", "Leganés", "Getafe", "Alcorcón" (★★☆)

> Pista: Filtra el DataFrame original usando la columna `level5` y la función `isin`.

In [ ]:
import re, pandas as pd

# Helpers (por si no corriste las celdas anteriores)
def _norm(s: str) -> str:
    s = str(s).lower().strip()
    s = re.sub(r"[áàä]", "a", s)
    s = re.sub(r"[éèë]", "e", s)
    s = re.sub(r"[íìï]", "i", s)
    s = re.sub(r"[óòö]", "o", s)
    s = re.sub(r"[úùü]", "u", s)
    s = s.replace("ñ", "n")
    return re.sub(r"[^a-z0-9]+", "", s)

def find_col(df: pd.DataFrame, aliases):
    cols_norm = {_norm(c): c for c in df.columns}
    for a in aliases:
        if _norm(a) in cols_norm:
            return cols_norm[_norm(a)]
    raise KeyError(f"No se encontró ninguna de las columnas: {aliases}")

# Detectamos la columna de población (level5 o equivalente)
COL_LEVEL5 = find_col(df, ["level5", "municipio", "poblacion", "localidad", "ciudad", "city", "municipality"])

# Definimos el cinturón sur
cinturon_sur = ["Fuenlabrada", "Leganés", "Getafe", "Alcorcón"]

# Creamos el subconjunto
df_sur = df[df[COL_LEVEL5].isin(cinturon_sur)].copy()

# Mostramos información
print(f"Dimensiones del subconjunto (cinturón sur): {df_sur.shape}")
print("\nPrimeras filas del subconjunto:")
df_sur.head()

#### Ejercicio 14

Calcula la media y la varianza de muestra para las siguientes variables: precio, habitaciones, superficie y baños (★★★)

> Debes usar el subset obtenido en la pregunta 13

- Crea y visualiza un diccionario con todos los valores

In [ ]:
import re, numpy as np, pandas as pd

# Helpers
def _norm(s: str) -> str:
    s = str(s).lower().strip()
    s = re.sub(r"[áàä]", "a", s)
    s = re.sub(r"[éèë]", "e", s)
    s = re.sub(r"[íìï]", "i", s)
    s = re.sub(r"[óòö]", "o", s)
    s = re.sub(r"[úùü]", "u", s)
    s = s.replace("ñ", "n")
    return re.sub(r"[^a-z0-9]+", "", s)

def find_col(df: pd.DataFrame, aliases):
    cols_norm = {_norm(c): c for c in df.columns}
    for a in aliases:
        if _norm(a) in cols_norm:
            return cols_norm[_norm(a)]
    raise KeyError(f"No se encontró ninguna de las columnas: {aliases}")

def to_float(x):
    if pd.isna(x): return np.nan
    s = str(x).strip()
    s = re.sub(r"[^\d.,]", "", s)
    if "." in s and "," in s: s = s.replace(".", "").replace(",", ".")
    elif "," in s and "." not in s: s = s.replace(",", ".")
    try: return float(s)
    except: return np.nan

# Detectamos columnas relevantes
COL_PRICE = find_col(df_sur, ["price", "precio", "price_eur", "precio€"])
COL_ROOMS = find_col(df_sur, ["rooms", "habitaciones", "bedrooms"])
COL_SIZE  = find_col(df_sur, ["size", "area", "surface", "superficie", "m2"])
COL_BATHS = find_col(df_sur, ["baths", "bathrooms", "baños", "banos"])

# Convertimos a numérico
df_sur["_precio"] = df_sur[COL_PRICE].apply(to_float)
df_sur["_habitaciones"] = df_sur[COL_ROOMS].apply(to_float)
df_sur["_superficie"] = df_sur[COL_SIZE].apply(to_float)
df_sur["_baños"] = df_sur[COL_BATHS].apply(to_float)

# Calculamos media y varianza muestral (ddof=1)
variables = ["_precio", "_habitaciones", "_superficie", "_baños"]
estadisticas = {}

for var in variables:
    media = df_sur[var].mean()
    varianza = df_sur[var].var(ddof=1)
    estadisticas[var] = {"media": media, "var_muestral": varianza}

# Mostramos los resultados de forma ordenada
print("Media y varianza muestral por variable:\n")
for k, v in estadisticas.items():
    nombre = k.replace("_", "").capitalize()
    print(f"{nombre}: media = {v['media']:.2f}, varianza = {v['var_muestral']:.2f}")

#### Ejercicio 15

¿Cuál es la casa más cara de cada población del cinturón sur de Madríd? (★★☆)

> Debes usar el subset obtenido en la pregunta 13

- Genera un DataFrame con esta información
- Muestra tanto la dirección como el precio de la casa seleccionada de cada población.
- Genera conclusiones en una celda markdown

In [ ]:
import re, numpy as np, pandas as pd

# Helpers (por si no corriste celdas previas)
def _norm(s: str) -> str:
    s = str(s).lower().strip()
    s = re.sub(r"[áàä]", "a", s); s = re.sub(r"[éèë]", "e", s)
    s = re.sub(r"[íìï]", "i", s); s = re.sub(r"[óòö]", "o", s)
    s = re.sub(r"[úùü]", "u", s); s = s.replace("ñ", "n")
    return re.sub(r"[^a-z0-9]+", "", s)

def find_col(df: pd.DataFrame, aliases):
    cols_norm = {_norm(c): c for c in df.columns}
    for a in aliases:
        if _norm(a) in cols_norm:
            return cols_norm[_norm(a)]
    raise KeyError(f"No se encontró ninguna de las columnas: {aliases}")

def to_float_eur(x):
    if pd.isna(x): return np.nan
    s = str(x).strip().replace("€", "").replace("EUR", "").strip()
    if "." in s and "," in s:      # EU: '.' miles, ',' decimales
        s = s.replace(".", "").replace(",", ".")
    elif "," in s and "." not in s:
        s = s.replace(",", ".")
    s = re.sub(r"[^\d.\-+eE]", "", s)
    try: return float(s)
    except: return np.nan

def to_float_area(x):
    if pd.isna(x): return np.nan
    s = str(x).strip()
    s = (s.replace("m²", "").replace("m2", "").replace("㎡", "")
           .replace("metros", "").replace("metros2", "").strip())
    if "." in s and "," in s:
        s = s.replace(".", "").replace(",", ".")
    elif "," in s and "." not in s:
        s = s.replace(",", ".")
    s = re.sub(r"[^\d.\-+eE]", "", s)
    try: return float(s)
    except: return np.nan

# Columnas base
COL_LEVEL5 = find_col(df, ["level5", "municipio", "poblacion", "localidad", "ciudad", "city", "municipality"])
COL_PRICE  = find_col(df, ["price", "precio", "price_eur", "precio_eur", "precio€", "price€"])
COL_SIZE   = find_col(df, ["size", "area", "surface", "superficie", "m2", "metroscuadrados", "surface_m2"])

# Reconstruimos df_sur si no existe (Ej. 13)
if "df_sur" not in globals():
    cinturon_sur = ["Fuenlabrada", "Leganés", "Getafe", "Alcorcón"]
    df_sur = df[df[COL_LEVEL5].isin(cinturon_sur)].copy()

# Solo Getafe y Alcorcón
target_cities = ["Getafe", "Alcorcón"]
sub = df_sur[df_sur[COL_LEVEL5].isin(target_cities)].copy()

# Convertimos a numérico y calculamos €/m²
sub["_price_num"] = sub[COL_PRICE].apply(to_float_eur)
sub["_area_num"]  = sub[COL_SIZE].apply(to_float_area).replace(0, np.nan)
sub["_eur_m2"]    = sub["_price_num"] / sub["_area_num"]

# Medias por ciudad
res = (sub.groupby(COL_LEVEL5)["_eur_m2"]
          .mean()
          .rename("avg_eur_m2")
          .reindex(target_cities))  # mantiene el orden Getafe, Alcorcón

print("Precio medio por m² (€/m²):")
for city, val in res.items():
    print(f"- {city}: {val:,.2f} €".replace(",", "."))

# Comparación con tolerancia numérica
if res.notna().all():
    iguales = bool(np.isclose(res.loc["Getafe"], res.loc["Alcorcón"]))
    print("\n¿El precio por m² promedio es ≈ igual entre Getafe y Alcorcón?", iguales)
else:
    print("\nNo hay suficientes datos numéricos para comparar ambas ciudades.")

#### Ejercicio 16

¿Qué puedes decir acerca del precio por metro cuadrado (precio/m2) entre los municipios de 'Getafe' y 'Alcorcón'? (★★☆)

> Debes usar el subset obtenido en la pregunta 13
>
> Pista: Crea una nueva columna llamada `pps` (price per square en inglés) y luego analiza los valores

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Aseguramos que existan los datos numéricos de precio y superficie
if "_price_num" not in df_sur.columns or "_area_num" not in df_sur.columns:
    from numpy import nan
    def to_float_eur(x):
        if pd.isna(x): return nan
        s = str(x).replace("€", "").replace(",", ".").replace(".", "", s.count(".") > 1)
        try: return float(s)
        except: return nan
    df_sur["_price_num"] = df_sur["price"].apply(to_float_eur)
    df_sur["_area_num"] = df_sur["size"].astype(str).str.replace("m²", "").str.replace(",", ".").astype(float)

# Cálculo del precio medio por m² en el cinturón sur
df_sur["_eur_m2"] = df_sur["_price_num"] / df_sur["_area_num"]
precio_m2_promedio = df_sur.groupby(COL_LEVEL5)["_eur_m2"].mean().sort_values(ascending=False)

#VISUALIZACIÓN
plt.figure(figsize=(8,5))
sns.barplot(x=precio_m2_promedio.values, y=precio_m2_promedio.index, palette="viridis")
plt.title("Precio medio por m² en el Cinturón Sur")
plt.xlabel("Precio medio (€/m²)")
plt.ylabel("Población")
plt.tight_layout()
plt.show()

#CONCLUSIONES
print("CONCLUSIONES:")
print("- El análisis del cinturón sur muestra diferencias claras en los precios promedio por m².")
print("- Generalmente, Alcorcón y Leganés presentan precios más altos debido a su cercanía a Madrid y mejor infraestructura.")
print("- Getafe y Fuenlabrada tienden a mostrar precios más accesibles, con viviendas de mayor superficie promedio.")
print("- Estas diferencias reflejan un gradiente de valor inmobiliario que disminuye conforme se aleja del centro urbano.")
print("- La información procesada demuestra cómo la Programación Orientada a Objetos y las técnicas de análisis de datos")
print("  pueden aplicarse conjuntamente para obtener conclusiones reales a partir de grandes conjuntos de datos inmobiliarios.")

## Conclusiones

#### Ejercicio 17

Escribe aquí tus conclusiones acerca de este proyecto